In [1]:
#import all necessary packages
import matplotlib.pyplot as plt
from math import sqrt
import skimage as sk
from skimage.color import rgb2gray
from skimage.feature import blob_dog, blob_log, blob_doh, canny
from skimage.filters import rank
from skimage.morphology import disk, square, binary_dilation, binary_erosion
from skimage.segmentation import watershed
from skimage.util import img_as_float, img_as_ubyte, invert
from skimage.measure import label, regionprops, regionprops_table
from skimage.filters import threshold_otsu, sobel
from scipy.ndimage import binary_fill_holes
from skimage.morphology import remove_small_objects
# turn on if using colab
# from google.colab import drive
from pathlib import Path
import colorthief
import pandas as pd
import numpy as np
import numpy.ma as ma #masking
from time import sleep
from PIL import Image
from colorthief import ColorThief




In [2]:
#function to mask out all images

def super_mask(folder_read, folder_write):
    """
    This function masks out sherds from the background and saves them in a chosen folder for a subsequent operation, like ColorThief
    
    Parameters
    ----------
    folder_read: str
        str folder to path to with images
    folder_write: str
        str of folder to save masked images
    Returns
    -------
     A "Done" statement
    
    
    """
    
    pathstr = [str(path) for path in (Path(folder_read).rglob('*.jpeg'))]
    folder_len = len(folder_read) 
    
    for path in pathstr:
        image = plt.imread(path)
        
        #binarize image
        im_gray = rgb2gray(image)
        thresh_val = threshold_otsu(im_gray)
        new_mask = im_gray > thresh_val
        sherd_mask = remove_small_objects(binary_fill_holes(new_mask, disk(1)), 300)*1 #Multiply by 1 to transform True/False to 0/1
        imcan = canny(im_gray)
        sherd_bi = remove_small_objects(binary_fill_holes(binary_erosion(binary_dilation(imcan, disk(10)), disk(10))),200)*1
        #Need to 'stack' the image to create a 3D array, because RGB images are 3D arrays
        if sherd_bi.sum() < sherd_mask.sum():
            mask3D = np.dstack((sherd_mask,sherd_mask,sherd_mask))
        else:
            mask3D = np.dstack((sherd_bi,sherd_bi,sherd_bi))
        #masked = ma.masked_array(image, mask=mask3D) Not sure why the mask thing doesn't work but taking the product will
        
        #multiplying the image by the 3Dmask to basically create a large 0,0,0 area for the background
        masked = image*mask3D
        
        #save the image in the specified folder
        masked_image = Image.fromarray(masked.astype(np.uint8))
        masked_image.save(f'{folder_write}/{path[folder_len:]}')
    print('Done!')
    

In [3]:
def inclusion_counter2(folder_path, blob_algorithm = 'all', max_sigma = 30, threshold = 0.4, scan_dpi = 600):
    threshold_doh = threshold/40
    dpcm = (scan_dpi*.3937)**2
    path_strs = [str(path) for path in (Path(folder_path).rglob('*.jpeg'))]
     # because path is object not string


    #image operations
    maxrad = []
    meanrad = []
    minrad = []
    inclusions = []
    std = []
    pct_25 = []
    pct_75 = []
    maxrad_reg = []
    meanrad_reg = []
    minrad_reg = []
    inclusions_reg = []
    std_reg = []
    pct_25_reg = []
    pct_75_reg = []
    names = []
    area = []
    maxrad_doh = []
    meanrad_doh = []
    minrad_doh = []
    inclusions_doh = []
    std_doh = []
    pct_25_doh = []
    pct_75_doh = []
    maxrad_dog = []
    meanrad_dog = []
    minrad_dog = []
    inclusions_dog = []
    std_dog = []
    pct_25_dog = []
    pct_75_dog = []
    maxrad_log = []
    meanrad_log = []
    minrad_log = []
    inclusions_log = []
    std_log = []
    pct_25_log = []
    pct_75_log = []
    
    data = pd.DataFrame()

    if blob_algorithm == 'doh':
        for path in path_strs:
            name = path
            names.append(name.rsplit(sep = '/')[-1]) #need to get a column in there with names from each ceramic might need to be part of top chunk
            im = plt.imread(name)
            image_gray = rgb2gray(im)
            blobs_doh = blob_doh(image_gray, max_sigma = max_sigma, threshold = threshold_doh)
            inclusions.append(len(blobs_doh))
            maxrad.append(np.max(blobs_doh[:,2]))
            meanrad.append(blobs_doh[:,2].mean())
            minrad.append(blobs_doh[:,2].min())
            std.append(np.std(blobs_doh[:,2]))
            pct_25.append(np.percentile(blobs_doh[:,2], 25))
            pct_75.append(np.percentile(blobs_doh[:,2], 75))
            imcan = canny(rgb2gray(im))
            sherd_bi = binary_fill_holes(binary_erosion(binary_dilation(imcan, disk(10)), disk(10)))
            sherd_bi = remove_small_objects(sherd_bi, 200)
            object_labels = label((sherd_bi))
            regions = regionprops_table(object_labels, properties = ['area'])
            tempdf = pd.DataFrame(regions)
            area.append(np.max(regions['area'])/dpcm)        
        data['Name']= names 
        data['Maximum Inclusion Radius'] = maxrad
        data['Average Inclusion Radius'] = meanrad
        data['Minimum Inclusion Radius'] = minrad
        data['Estimated Total Inclusions']= inclusions
        data['Standard Deviation'] = std
        data['25th Percentile'] = pct_25
        data['75th Percentile'] = pct_75
        data['Sherd Area sq cm'] = area
        data['Inclusions/ Sherd Area']= data['Estimated Total Inclusions']/data['Sherd Area sq cm']
            
    elif blob_algorithm == 'dog':
        for path in path_strs:
            name = path
            names.append(name.rsplit(sep = '/')[-1]) #need to get a column in there with names from each ceramic might need to be part of top chunk
            im = plt.imread(name)
            image_gray = rgb2gray(im)
            blobs_dog = blob_dog(image_gray, max_sigma = max_sigma, threshold = threshold)
            inclusions.append(len(blobs_dog))
            maxrad.append(blobs_dog[:,2].max())
            meanrad.append(blobs_dog[:,2].mean())
            minrad.append(blobs_dog[:,2].min())
            std.append(np.std(blobs_dog[:,2]))
            pct_25.append(np.percentile(blobs_dog[:,2], 25))
            pct_75.append(np.percentile(blobs_dog[:,2], 75))
            imcan = canny(rgb2gray(im))
            sherd_bi = binary_fill_holes(binary_erosion(binary_dilation(imcan, disk(10)), disk(10)))
            sherd_bi = remove_small_objects(sherd_bi, 200)
            object_labels = label((sherd_bi))
            regions = regionprops_table(object_labels, properties = ['area'])
            tempdf = pd.DataFrame(regions)
            area.append(np.max(regions['area'])/dpcm)      
        data['Name']= names 
        data['Maximum Inclusion Radius'] = maxrad
        data['Average Inclusion Radius'] = meanrad
        data['Minimum Inclusion Radius'] = minrad
        data['Estimated Total Inclusions']= inclusions
        data['Standard Deviation'] = std
        data['25th Percentile'] = pct_25
        data['75th Percentile'] = pct_75
        data['Sherd Area sq cm'] = area
        data['Inclusions/ Sherd Area']= data['Estimated Total Inclusions']/data['Sherd Area sq cm']
            

    elif blob_algorithm == 'log':
        for path in path_strs:
            name = path
            names.append(name.rsplit(sep = '/')[-1]) #need to get a column in there with names from each ceramic might need to be part of top chunk
            im = plt.imread(name)
            image_gray = rgb2gray(im)
            blobs_log = blob_log(image_gray, max_sigma = max_sigma, num_sigma = 10, threshold = threshold)
            blobs_log[:,2] = blobs_log[:, 2] * sqrt(2)
            inclusions.append(len(blobs_log))
            maxrad.append(blobs_log[:,2].max())
            meanrad.append(blobs_log[:,2].mean())
            minrad.append(blobs_log[:,2].min())
            std.append(np.std(blobs_log[:,2]))
            pct_25.append(np.percentile(blobs_log[:,2], 25))
            pct_75.append(np.percentile(blobs_log[:,2], 75))
            imcan = canny(rgb2gray(im))
            sherd_bi = binary_fill_holes(binary_erosion(binary_dilation(imcan, disk(10)), disk(10)))
            sherd_bi = remove_small_objects(sherd_bi, 200)
            object_labels = label((sherd_bi))
            regions = regionprops_table(object_labels, properties = ['area'])
            area.append(regions['area'].max()/dpcm)       
        data['Name']= names 
        data['Maximum Inclusion Radius'] = maxrad
        data['Average Inclusion Radius'] = meanrad
        data['Minimum Inclusion Radius'] = minrad
        data['Estimated Total Inclusions']= inclusions
        data['Standard Deviation'] = std
        data['25th Percentile'] = pct_25
        data['75th Percentile'] = pct_75
        data['Sherd Area sq cm'] = area
        data['Inclusions/ Sherd Area']= data['Estimated Total Inclusions']/data['Sherd Area sq cm']
   

    if blob_algorithm == 'region':
        for path in path_strs:
            name = path
            names.append(name.rsplit(sep = '/')[-1]) #need to get a column in there with names from each ceramic might need to be part of top chunk
            im = plt.imread(name)
            image_gray = rgb2gray(im)
            thresholda = threshold_otsu(image_gray)
            mask = image_gray > thresholda
            object_labels = label(mask)
            regions = regionprops_table(object_labels, properties = ['area', 'eccentricity', 'major_axis_length', 'orientation'])
            df = pd.DataFrame(regions)
            df = df[df['area'] > 3]
            inclusions.append(len(df))
            maxrad.append(np.sort(df['major_axis_length'])[-2])
            meanrad.append(np.mean(df['major_axis_length']))
            minrad.append(np.min(df['major_axis_length']))
            std.append(np.std(df['major_axis_length']))
            pct_25.append(np.percentile(df['major_axis_length'], 25))
            pct_75.append(np.percentile(df['major_axis_length'], 75))
            area.append(regions['area'].max()/dpcm)       
        data['Name']= names 
        data['Maximum Inclusion Radius'] = maxrad
        data['Average Inclusion Radius'] = meanrad
        data['Minimum Inclusion Radius'] = minrad
        data['Estimated Total Inclusions']= inclusions
        data['Standard Deviation'] = std
        data['25th Percentile'] = pct_25
        data['75th Percentile'] = pct_75
        data['Sherd Area sq cm'] = area
        data['Inclusions/ Sherd Area']= data['Estimated Total Inclusions']/data['Sherd Area sq cm']
        
 
    elif blob_algorithm == 'all':
        for path in path_strs:
            name = path
            names.append(name.rsplit(sep = '/')[-1]) #need to get a column in there with names from each ceramic might need to be part of top chunk
            im = plt.imread(name)
            image_gray = rgb2gray(im)
            blobs_doh = blob_doh(image_gray, max_sigma = max_sigma, threshold = threshold_doh)
            blobs_dog = blob_dog(image_gray, max_sigma = max_sigma, threshold = threshold)
            blobs_dog[:,2] = blobs_dog[:, 2] * sqrt(2)
            blobs_log = blob_log(image_gray, max_sigma = max_sigma, num_sigma = 10, threshold = threshold)
            blobs_log[:,2] = blobs_log[:, 2] * sqrt(2)
            thresholda = threshold_otsu(image_gray)
            mask = image_gray > thresholda
            object_labels = label(mask)
            regions = regionprops_table(object_labels, properties = ['area', 'eccentricity', 'major_axis_length', 'orientation'])
            df = pd.DataFrame(regions)
            df = df[df['area'] > 3]
            inclusions_reg.append(len(df))
            maxrad_reg.append(np.sort(df['major_axis_length'])[-2])
            meanrad_reg.append(np.mean(df['major_axis_length']))
            minrad_reg.append(np.min(df['major_axis_length']))
            std_reg.append(np.std(df['major_axis_length']))
            pct_25_reg.append(np.percentile(df['major_axis_length'], 25))
            pct_75_reg.append(np.percentile(df['major_axis_length'], 75)) 
            inclusions_doh.append(len(blobs_doh))
            maxrad_doh.append(np.max(blobs_doh[:,2]))
            meanrad_doh.append(blobs_doh[:,2].mean())
            minrad_doh.append(blobs_doh[:,2].min())
            std_doh.append(np.std(blobs_doh[:,2]))
            pct_25_doh.append(np.percentile(blobs_doh[:,2], 25))
            pct_75_doh.append(np.percentile(blobs_doh[:,2], 75))
            inclusions_dog.append(len(blobs_dog))
            maxrad_dog.append(blobs_dog[:,2].max())
            meanrad_dog.append(blobs_dog[:,2].mean())
            minrad_dog.append(blobs_dog[:,2].min())
            std_dog.append(np.std(blobs_dog[:,2]))
            pct_25_dog.append(np.percentile(blobs_dog[:,2], 25))
            pct_75_dog.append(np.percentile(blobs_dog[:,2], 75))
            inclusions_log.append(len(blobs_log))
            maxrad_log.append(blobs_log[:,2].max())
            meanrad_log.append(blobs_log[:,2].mean())
            minrad_log.append(blobs_log[:,2].min())
            std_log.append(np.std(blobs_log[:,2]))
            pct_25_log.append(np.percentile(blobs_log[:,2], 25))
            pct_75_log.append(np.percentile(blobs_log[:,2], 75))
            imcan = canny(rgb2gray(im))
            sherd_bi = binary_fill_holes(binary_erosion(binary_dilation(imcan, disk(10)), disk(10)))
            sherd_bi = remove_small_objects(sherd_bi, 200)
            object_labels = label((sherd_bi))
            regions = regionprops_table(object_labels, properties = ['area'])
            area.append((regions['area'].max())/dpcm) #dpi to get square centimeters       
        data['Name']= names
        data['Estimated Total Inclusions DoH']= inclusions_doh
        data['Estimated Total Inclusions DoG']= inclusions_dog
        data['Estimated Total Inclusions LoG']= inclusions_log
        data['Estimated Total Inclusions reg']= inclusions_reg
        
        data['Maximum Inclusion Radius DoH'] = maxrad_doh
        data['Maximum Inclusion Radius DoG'] = maxrad_dog
        data['Maximum Inclusion Radius LoG'] = maxrad_log
        data['Maximum Inclusion Radius reg'] = maxrad_reg
        
        
        
        data['Standard Deviation DoH'] = std_doh
        data['Standard Deviation DoG'] = std_dog
        data['Standard Deviation LoG'] = std_log
        data['Standard Deviation reg'] = std_reg
        
        data['25th Percentile DoH'] = pct_25_doh
        data['25th Percentile DoG'] = pct_25_dog
        data['25th Percentile LoG'] = pct_25_log
        data['25th Percentile reg'] = pct_25_reg
        
        
        data['Average Inclusion Radius DoH'] = meanrad_doh
        data['Average Inclusion Radius DoG'] = meanrad_dog
        data['Average Inclusion Radius LoG'] = meanrad_log
        data['Average Inclusion Radius reg'] = meanrad_reg
        
        
        
        
        data['75th Percentile DoG'] = pct_75_dog
        data['75th Percentile DoH'] = pct_75_doh
        data['75th Percentile LoG'] = pct_75_log
        data['75th Percentile reg'] = pct_75_reg
        
        
        data['Minimum Inclusion Radius DoH'] = minrad_doh
        data['Minimum Inclusion Radius DoG'] = minrad_dog
        data['Minimum Inclusion Radius LoG'] = minrad_log
        data['Minimum Inclusion Radius reg'] = minrad_reg
        
        
        
        
        data['Sherd Area sq cm'] = area
        data['Inclusions/ Sherd Area DoH']= data['Estimated Total Inclusions DoH']/data['Sherd Area sq cm']
        data['Inclusions/ Sherd Area DoG']= data['Estimated Total Inclusions DoG']/data['Sherd Area sq cm']
        data['Inclusions/ Sherd Area LoG']= data['Estimated Total Inclusions LoG']/data['Sherd Area sq cm']
        data['Inclusions/ Sherd Area reg']= data['Estimated Total Inclusions reg']/data['Sherd Area sq cm']
            
    
    
     
    
    

    return data

In [4]:
def pull_colors(folder_read):
    """
    Finds the most common RGB value using the ColorThief function in each image and generates a pandas database with each color value as its own column. 
    This function assumes image backgrounds are all uniform and will be the most common color, therefore, it will actually be selecting the second most common RGB value
    
    Parameters
    ----------
    
    folder_read: str
        str of folder to read in
    
    Returns
    -------
    color_df: pandas dataframe
        4 columns with the file name, and seperated rgb values
    """
    
    pathstr = [str(path) for path in (Path(folder_read).rglob('*.jpeg'))]
    folder_len = len(folder_read)
    R = []
    G = []
    B = []
    R2 = []
    G2 = []
    B2  = []
    names = []
    df = pd.DataFrame()
   
    for path in pathstr:
        thief = ColorThief(path)
        color = thief.get_palette(color_count=2)
        rgb = list(color[1]) #pulling the second color, the first will be the black background
        rgb2 = list(color[2])
        #splitting out each RGB value allows for us to perform a cluster analysis
        R.append(rgb[0]) 
        G.append(rgb[1])
        B.append(rgb[2])
        
        R2.append(rgb2[0]) 
        G2.append(rgb2[1])
        B2.append(rgb2[2])
        
        namy = path[folder_len:]
        names.append(namy)
    df['name'] = names
    df['red'] = R
    df['green'] = G
    df['blue'] = B
    df['red2'] = R
    df['green2'] = G
    df['blue2'] = B
    
    return(df)
    

In [5]:
folder_read = 'Ceramics_Images'
folder_write = 'zorro'
super_mask(folder_read, folder_write)

Done!


In [6]:
super_mask(folder_read, folder_write)
inclusions = inclusion_counter2(folder_write)
colors = pull_colors(folder_write)
inclusions

Done!


,Name,Estimated Total Inclusions DoH,Estimated Total Inclusions DoG,Estimated Total Inclusions LoG,Estimated Total Inclusions reg,Maximum Inclusion Radius DoH,Maximum Inclusion Radius DoG,Maximum Inclusion Radius LoG,Maximum Inclusion Radius reg,Standard Deviation DoH,...,75th Percentile reg,Minimum Inclusion Radius DoH,Minimum Inclusion Radius DoG,Minimum Inclusion Radius LoG,Minimum Inclusion Radius reg,Sherd Area sq cm,Inclusions/ Sherd Area DoH,Inclusions/ Sherd Area DoG,Inclusions/ Sherd Area LoG,Inclusions/ Sherd Area reg
0,zorro\Ceramic_01.jpeg,780,137,396,96,30.000000,37.962506,5.971124,26.030030,2.148307,...,6.978121,1.0,2.262742,1.414214,2.000000,12.285473,63.489621,11.151382,32.233192,7.814107
1,zorro\Ceramic_02.jpeg,307,103,128,49,23.555556,37.962506,5.971124,22.054930,2.684022,...,8.620054,1.0,3.620387,1.414214,2.000000,11.947838,25.695026,8.620807,10.713236,4.101161
2,zorro\Ceramic_03.jpeg,402,37,119,65,7.444444,37.962506,1.414214,29.639274,1.320123,...,6.011487,1.0,9.268190,1.414214,2.000000,21.572695,18.634667,1.715131,5.516232,3.013068
3,zorro\Ceramic_04.jpeg,74,40,39,5,4.222222,37.962506,1.414214,5.656854,1.472776,...,5.656854,1.0,9.268190,1.414214,2.000000,23.332968,3.171478,1.714313,1.671455,0.214289
4,zorro\Ceramic_05.jpeg,30,30,2,3,7.444444,37.962506,1.414214,3.752200,1.921362,...,793.448377,1.0,23.726566,1.414214,3.464102,13.445260,2.231270,2.231270,0.148751,0.223127
5,zorro\Ceramic_06.jpeg,60,28,3,22,10.666667,37.962506,1.414214,12.092680,1.751762,...,6.093780,1.0,5.792619,1.414214,2.828427,12.870510,4.661820,2.175516,0.233091,1.709334
6,zorro\Ceramic_07.jpeg,749,87,349,116,17.111111,37.962506,1.414214,34.518136,1.947267,...,6.928452,1.0,3.620387,1.414214,2.000000,11.359700,65.934839,7.658653,30.722642,10.211537
7,zorro\Ceramic_08.jpeg,404,113,154,109,23.555556,37.962506,1.414214,39.177444,2.308191,...,8.281471,1.0,3.620387,1.414214,2.000000,12.335293,32.751552,9.160706,12.484502,8.836433
8,zorro\Ceramic_09.jpeg,481,79,125,44,17.111111,37.962506,5.971124,32.046812,1.766964,...,9.474674,1.0,3.620387,1.414214,2.000000,6.929189,69.416488,11.401045,18.039628,6.349949
9,zorro\Ceramic_10.jpeg,276,73,57,27,17.111111,37.962506,1.414214,44.074513,1.690336,...,8.106559,1.0,3.620387,1.414214,2.828427,6.108346,45.184084,11.950863,9.331496,4.420182


In [7]:
colors

,name,red,green,blue,red2,green2,blue2
0,\Ceramic_01.jpeg,174,170,167,174,170,167
1,\Ceramic_02.jpeg,169,165,161,169,165,161
2,\Ceramic_03.jpeg,157,139,126,157,139,126
3,\Ceramic_04.jpeg,147,126,110,147,126,110
4,\Ceramic_05.jpeg,179,125,106,179,125,106
5,\Ceramic_06.jpeg,180,127,108,180,127,108
6,\Ceramic_07.jpeg,174,160,147,174,160,147
7,\Ceramic_08.jpeg,171,158,145,171,158,145
8,\Ceramic_09.jpeg,170,156,152,170,156,152
9,\Ceramic_10.jpeg,167,155,153,167,155,153


In [8]:
inclusions['red'] = colors['red']
inclusions['green'] = colors['green']
inclusions['blue']=colors['blue']

In [9]:
inclusions.columns

Index(['Name', 'Estimated Total Inclusions DoH',
       'Estimated Total Inclusions DoG', 'Estimated Total Inclusions LoG',
       'Estimated Total Inclusions reg', 'Maximum Inclusion Radius DoH',
       'Maximum Inclusion Radius DoG', 'Maximum Inclusion Radius LoG',
       'Maximum Inclusion Radius reg', 'Standard Deviation DoH',
       'Standard Deviation DoG', 'Standard Deviation LoG',
       'Standard Deviation reg', '25th Percentile DoH', '25th Percentile DoG',
       '25th Percentile LoG', '25th Percentile reg',
       'Average Inclusion Radius DoH', 'Average Inclusion Radius DoG',
       'Average Inclusion Radius LoG', 'Average Inclusion Radius reg',
       '75th Percentile DoG', '75th Percentile DoH', '75th Percentile LoG',
       '75th Percentile reg', 'Minimum Inclusion Radius DoH',
       'Minimum Inclusion Radius DoG', 'Minimum Inclusion Radius LoG',
       'Minimum Inclusion Radius reg', 'Sherd Area sq cm',
       'Inclusions/ Sherd Area DoH', 'Inclusions/ Sherd Area DoG',

In [10]:
inclusions

,Name,Estimated Total Inclusions DoH,Estimated Total Inclusions DoG,Estimated Total Inclusions LoG,Estimated Total Inclusions reg,Maximum Inclusion Radius DoH,Maximum Inclusion Radius DoG,Maximum Inclusion Radius LoG,Maximum Inclusion Radius reg,Standard Deviation DoH,...,Minimum Inclusion Radius LoG,Minimum Inclusion Radius reg,Sherd Area sq cm,Inclusions/ Sherd Area DoH,Inclusions/ Sherd Area DoG,Inclusions/ Sherd Area LoG,Inclusions/ Sherd Area reg,red,green,blue
0,zorro\Ceramic_01.jpeg,780,137,396,96,30.000000,37.962506,5.971124,26.030030,2.148307,...,1.414214,2.000000,12.285473,63.489621,11.151382,32.233192,7.814107,174,170,167
1,zorro\Ceramic_02.jpeg,307,103,128,49,23.555556,37.962506,5.971124,22.054930,2.684022,...,1.414214,2.000000,11.947838,25.695026,8.620807,10.713236,4.101161,169,165,161
2,zorro\Ceramic_03.jpeg,402,37,119,65,7.444444,37.962506,1.414214,29.639274,1.320123,...,1.414214,2.000000,21.572695,18.634667,1.715131,5.516232,3.013068,157,139,126
3,zorro\Ceramic_04.jpeg,74,40,39,5,4.222222,37.962506,1.414214,5.656854,1.472776,...,1.414214,2.000000,23.332968,3.171478,1.714313,1.671455,0.214289,147,126,110
4,zorro\Ceramic_05.jpeg,30,30,2,3,7.444444,37.962506,1.414214,3.752200,1.921362,...,1.414214,3.464102,13.445260,2.231270,2.231270,0.148751,0.223127,179,125,106
5,zorro\Ceramic_06.jpeg,60,28,3,22,10.666667,37.962506,1.414214,12.092680,1.751762,...,1.414214,2.828427,12.870510,4.661820,2.175516,0.233091,1.709334,180,127,108
6,zorro\Ceramic_07.jpeg,749,87,349,116,17.111111,37.962506,1.414214,34.518136,1.947267,...,1.414214,2.000000,11.359700,65.934839,7.658653,30.722642,10.211537,174,160,147
7,zorro\Ceramic_08.jpeg,404,113,154,109,23.555556,37.962506,1.414214,39.177444,2.308191,...,1.414214,2.000000,12.335293,32.751552,9.160706,12.484502,8.836433,171,158,145
8,zorro\Ceramic_09.jpeg,481,79,125,44,17.111111,37.962506,5.971124,32.046812,1.766964,...,1.414214,2.000000,6.929189,69.416488,11.401045,18.039628,6.349949,170,156,152
9,zorro\Ceramic_10.jpeg,276,73,57,27,17.111111,37.962506,1.414214,44.074513,1.690336,...,1.414214,2.828427,6.108346,45.184084,11.950863,9.331496,4.420182,167,155,153


In [11]:
saveall=inclusions.loc[:,['Estimated Total Inclusions DoH',
       'Estimated Total Inclusions DoG', 'Estimated Total Inclusions LoG',
       'Estimated Total Inclusions reg', 'Maximum Inclusion Radius DoH',
       'Maximum Inclusion Radius reg', 'Standard Deviation DoH',
       'Standard Deviation DoG', 'Standard Deviation LoG',
       'Standard Deviation reg',
       'Average Inclusion Radius DoH', 'Average Inclusion Radius DoG',
       'Average Inclusion Radius LoG', 'Average Inclusion Radius reg',
       'Minimum Inclusion Radius DoG',
       'Minimum Inclusion Radius reg',
       'Inclusions/ Sherd Area DoH', 'Inclusions/ Sherd Area DoG',
       'Inclusions/ Sherd Area LoG', 'Inclusions/ Sherd Area reg', 'red',
       'green', 'blue', 'red2', 'green2', 'blue2']]

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['red2', 'green2', 'blue2'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [ ]:
saveall.to_csv('final_data')

In [ ]:
saveall